## Importing dependencies

In [1]:
import json
import asyncio
import aiohttp
import time
import os

from setup_nyc_payroll import download_data, read_from_file

## Reading payroll data

In [4]:
FILENAME = 'nyc_payroll.json'
payroll_data = read_from_file(FILENAME)

## Viewing data
